In [1]:
epochs= 25

# SplitNN for Vertically Partitioned Data

<b>What is Vertically Partitioned Data?</b> Data is said to be vertically partitioned when several organizations own different attributes or modalities of information for the same set of entities.

<b>Why use Partitioned Data?</b> Partition allows for orgnizations holding different modalities of data to learn distributed models without data sharing. Partitioning scheme is traditionally used to reduce the size of data by splitting and distribute to each client.
 
<b>Description</b>This configuration allows for multiple clients holding different modalities of data to learn distributed models without data sharing. As a concrete example we walkthrough the case where radiology centers collaborate with pathology test centers and a server for disease diagnosis. Radiology centers holding imaging data modalities train a partial model upto the cut layer. In the same way the pathology test center having patient test results trains a partial model upto its own cut layer. The outputs at the cut layer from both these centers are then concatenated and sent to the disease diagnosis server that trains the rest of the model. This process is continued back and forth to complete the forward and backward propagations in order to train the distributed deep learning model without sharing each others raw data. In this tutorial, we split a single flatten image into two segments to mimic different modalities of data, you can also split it into arbitrary number.

<img src="images/config_1.png" width="40%">

In this tutorial, we demonstrate the SplitNN architecture with 2 segments[[1](https://arxiv.org/abs/1812.00564)].This time:

- <b>$Client_{1}$</b>
    - Has Model Segment 1
    - Has the handwritten images segment 1
- <b>$Client_{2}$</b>
    - Has model Segment 1
    - Has the handwritten images segment 2
- <b>$Server$</b> 
    - Has Model Segment 2
    - Has the image labels
    
Author:
- Abbas Ismail - github：[@abbas5253](https://github.com/abbas5253)

In [2]:
import torch
from torchvision import datasets, transforms
from torch import nn, optim
import syft as sy
import numpy as np

hook = sy.TorchHook(torch)

from distribute_data import Distribute_MNIST

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/home/ab_53/miniconda3/envs/PySyft/lib/python3.7/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.3.so'


In [3]:
# Data preprocessing
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])
trainset = datasets.MNIST('mnist', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# create some workers
client_1 = sy.VirtualWorker(hook, id="client_1")
client_2 = sy.VirtualWorker(hook, id="client_2")

server = sy.VirtualWorker(hook, id= "server") 

data_owners = (client_1, client_2)
model_locations = [client_1, client_2, server]

#Split each image and send one part to client_1, and other to client_2
distributed_trainloader = Distribute_MNIST(data_owners=data_owners, data_loader=trainloader)

In [4]:
input_size= [28*14, 28*14]
hidden_sizes= {"client_1": [32, 64], "client_2":[32, 64], "server":[128, 64]}

#create model segment for each worker
models = {
    "client_1": nn.Sequential(
                nn.Linear(input_size[0], hidden_sizes["client_1"][0]),
                nn.ReLU(),
                nn.Linear(hidden_sizes["client_1"][0], hidden_sizes["client_1"][1]),
                nn.ReLU(),
    ),
    "client_2":  nn.Sequential(
                nn.Linear(input_size[1], hidden_sizes["client_2"][0]),
                nn.ReLU(),
                nn.Linear(hidden_sizes["client_2"][0], hidden_sizes["client_2"][1]),
                nn.ReLU(),
    ),
    "server": nn.Sequential(
                nn.Linear(hidden_sizes["server"][0], hidden_sizes["server"][1]),
                nn.ReLU(),
                nn.Linear(hidden_sizes["server"][1], 10),
                nn.LogSoftmax(dim=1)
    )
}



# Create optimisers for each segment and link to their segment
optimizers = [
    optim.SGD(models[location.id].parameters(), lr=0.05,)
    for location in model_locations
]


#send model segement to each client and server
for location in model_locations:
    models[location.id].send(location)

In [5]:
def predict(data_pointer, models, data_owners, server):
    
    #individual client's output upto their respective cut layer
    client_output = {}
    
    #outputs that is moved to server and subjected to concatenate for server input
    remote_outputs = []
    
    #iterate over each client and pass thier inputs to respective model segment and send outputs to server
    for owner in data_owners:
        client_output[owner.id] = models[owner.id](data_pointer[owner.id].reshape([-1, 14*28]))
        remote_outputs.append(
            client_output[owner.id].move(server)
        )
    
    #concat outputs from all clients at server's location
    server_input = torch.cat(remote_outputs, 1)
    
    #pass concatenated output from server's model segment
    pred = models["server"](server_input)
    
    return pred

In [6]:
def train(data_pointer, target, data_owners, models, optimizers, server):
    
    #make grads zero
    for opt in optimizers:
        opt.zero_grad()
    
    #predict the output
    pred = predict(data_pointer, models, data_owners, server)
    
    #calculate loss
    criterion = nn.NLLLoss()
    loss = criterion(pred, target.reshape(-1, 64)[0])
    
    #backpropagate
    loss.backward()
    
    #optimization step
    for opt in optimizers:
        opt.step()
        
    return loss.detach().get()

In [7]:
#training
for i in range(epochs):
    running_loss = 0
    
    #iterate over each datapoints 
    for data_ptr, label in distributed_trainloader:
        
        #send labels to server's location for training
        label = label.send(server)
        
        loss = train(data_ptr, label, data_owners, models, optimizers, server)
        running_loss += loss

    else:
        print("Epoch {} - Training loss: {}".format(i, running_loss/len(trainloader)))

Epoch 0 - Training loss: 2.074364185333252
Epoch 1 - Training loss: 1.4119279384613037
Epoch 2 - Training loss: 1.0729103088378906
Epoch 3 - Training loss: 0.9095255136489868
Epoch 4 - Training loss: 0.8234174251556396
Epoch 5 - Training loss: 0.7714390754699707
Epoch 6 - Training loss: 0.7359529733657837
Epoch 7 - Training loss: 0.7096782922744751
Epoch 8 - Training loss: 0.6890704035758972
Epoch 9 - Training loss: 0.6721634268760681
Epoch 10 - Training loss: 0.65777188539505
Epoch 11 - Training loss: 0.6452123522758484
Epoch 12 - Training loss: 0.6340039968490601
Epoch 13 - Training loss: 0.6238173246383667
Epoch 14 - Training loss: 0.6144205331802368
Epoch 15 - Training loss: 0.6057182550430298
Epoch 16 - Training loss: 0.5975443124771118
Epoch 17 - Training loss: 0.5897234082221985
Epoch 18 - Training loss: 0.5822254419326782
Epoch 19 - Training loss: 0.5749425888061523
Epoch 20 - Training loss: 0.5678632259368896
Epoch 21 - Training loss: 0.5609995722770691
Epoch 22 - Training los

In [21]:
def test(model, dataloader, dataset_name):
    correct = 0
    with torch.no_grad():
        for data_ptr, label in dataloader:
            output = predict(data_ptr, models, data_owners, server).get()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(label.data.view_as(pred)).sum()

    print("{}: Accuracy {}/{} ({:.0f}%)".format(dataset_name, 
                                                correct,
                                                len(dataloader)* 64, 
                                                100. * correct / (len(dataloader) * 64) ))

In [22]:
#prepare and distribute test dataset
testset = datasets.MNIST('mnist', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)
distributed_testloader = Distribute_MNIST(data_owners=data_owners, data_loader=testloader)

#Accuracy on train and test sets
test(models, distributed_trainloader, "Train set")
test(models, distributed_testloader, "Test set")

Train set: Accuracy 49885/59968 (83%)
Test set: Accuracy 8290/9984 (83%)
